In [8]:
import numpy as np
import numpy.testing as npt
from scipy.linalg import expm
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
from matplotlib.collections import PatchCollection
import scipy.spatial, scipy.linalg
import scipy.sparse.linalg
from scipy.misc import logsumexp
from scipy.cluster.hierarchy import linkage, dendrogram
import itertools
import zipfile,io
import pandas as pd
import re
import sheet3 as imp
%matplotlib inline

In [2]:
unzip = zipfile.ZipFile("sheet3_datasets.zip", 'r')          
fl = unzip.namelist() 

In [3]:
bxte=np.array(pd.read_table(unzip.open(fl[5]),header=None,delim_whitespace=True))
bxtr=np.array(pd.read_table(unzip.open(fl[6]),header=None,delim_whitespace=True))
byte=np.array(pd.read_table(unzip.open(fl[7]),header=None,delim_whitespace=True))
bytr=np.array(pd.read_table(unzip.open(fl[8]),header=None,delim_whitespace=True))

dxte=np.array(pd.read_table(unzip.open(fl[9]),header=None,delim_whitespace=True))
dxtr=np.array(pd.read_table(unzip.open(fl[10]),header=None,delim_whitespace=True))
dyte=np.array(pd.read_table(unzip.open(fl[11]),header=None,delim_whitespace=True))
dytr=np.array(pd.read_table(unzip.open(fl[12]),header=None,delim_whitespace=True))

fxte=np.array(pd.read_table(unzip.open(fl[13]),header=None,delim_whitespace=True))
fxtr=np.array(pd.read_table(unzip.open(fl[14]),header=None,delim_whitespace=True))
fyte=np.array(pd.read_table(unzip.open(fl[15]),header=None,delim_whitespace=True))
fytr=np.array(pd.read_table(unzip.open(fl[16]),header=None,delim_whitespace=True))

ixte=np.array(pd.read_table(unzip.open(fl[17]),header=None,delim_whitespace=True))
ixtr=np.array(pd.read_table(unzip.open(fl[18]),header=None,delim_whitespace=True))
iyte=np.array(pd.read_table(unzip.open(fl[19]),header=None,delim_whitespace=True))
iytr=np.array(pd.read_table(unzip.open(fl[20]),header=None,delim_whitespace=True))

rxte=np.array(pd.read_table(unzip.open(fl[21]),header=None,delim_whitespace=True))
rxtr=np.array(pd.read_table(unzip.open(fl[22]),header=None,delim_whitespace=True))
ryte=np.array(pd.read_table(unzip.open(fl[23]),header=None,delim_whitespace=True))
rytr=np.array(pd.read_table(unzip.open(fl[24]),header=None,delim_whitespace=True))

In [4]:
print(bxte.shape)
print(bxtr.shape)
print(byte.shape)
print(bytr.shape)

print(dxte.shape)
print(dxtr.shape)
print(dyte.shape)
print(dytr.shape)

print(fxte.shape)
print(fxtr.shape)
print(fyte.shape)
print(fytr.shape)

print(ixte.shape)
print(ixtr.shape)
print(iyte.shape)
print(iytr.shape)

print(rxte.shape)
print(rxtr.shape)
print(ryte.shape)
print(rytr.shape)

(2, 4900)
(2, 400)
(1, 4900)
(1, 400)
(8, 300)
(8, 468)
(1, 300)
(1, 468)
(9, 400)
(9, 666)
(1, 400)
(1, 666)
(18, 1010)
(18, 1300)
(1, 1010)
(1, 1300)
(20, 7000)
(20, 400)
(1, 7000)
(1, 400)


In [5]:
params_gaus = { 'kernel': ['gaussian'], 'kernelparameter': np.logspace(-2,2,10), 'regularization': [0]}
params_pol = { 'kernel': ['polynomial'], 'kernelparameter': np.logspace(-2,2,10), 'regularization': [0]}
params_lin = { 'kernel': ['linear'], 'kernelparameter': np.logspace(-2,2,10), 'regularization': [0]}

In [6]:
def zerooneloss(y_true, y_pred):
    assert(len(y_true) == len(y_pred))
    pred = np.array(y_pred[:,0])
    true = np.array(y_true[:,0])
    true[np.where(true==-1)]=0
    b = np.mean(pred)
    pred[np.where(pred>=b)] = 1
    pred[np.where(pred<b)] = 0
    right = (true == pred).astype(np.int64).sum()
    loss = (len(true)-right)/len(true)
    return loss

In [7]:
bkrr = imp.cv(bxtr.T, bytr[0], imp.krr, params_gaus,loss_function=zerooneloss)

/home/ubthiro/Dokumente/WirtsInf/ML_Lab_Course

/Ubung/03/sheet3.py:63: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if(Y==None):


In [9]:
bYY= bkrr.predict(bxte.T)
print(zerooneloss(byte.T, bYY))
print(bYY)

/home/ubthiro/Dokumente/WirtsInf/ML_Lab_Course

/Ubung/03/sheet3.py:63: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if(Y==None):


0.142040816327
[[ 0.76685801]
 [-0.28244652]
 [-0.53059941]
 ..., 
 [ 1.125911  ]
 [-0.5789667 ]
 [-0.15712699]]


In [10]:
print(bkrr.kernel)
print(bkrr.kernelparameter)
print(bkrr.regularization)

['gaussian']
4.64158883361
0.0001


In [11]:
bYY[:20]

array([[ 0.76685801],
       [-0.28244652],
       [-0.53059941],
       [ 0.4042691 ],
       [ 0.24033782],
       [ 1.2342825 ],
       [-0.18544445],
       [-0.15050454],
       [ 0.14032427],
       [ 0.36723597],
       [ 0.07509364],
       [-0.9024896 ],
       [-0.70597787],
       [ 0.20374358],
       [ 0.03482679],
       [ 0.62869663],
       [-0.06196566],
       [ 0.74736759],
       [ 0.25730781],
       [-0.77142069]])

In [12]:
byte.T[:20]

array([[ 1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [ 1.],
       [ 1.],
       [-1.],
       [ 1.],
       [ 1.],
       [-1.]])

In [137]:
        if(self.c==0):
            D, U =np.linalg.eigh(K)
            c = np.logspace(-2,2,10)
            cc = c.reshape(len(c),1,1)
            br = np.ones((len(c),1,1))
            I = np.eye(len(K))*br
            KK=br*K
            KCI=KK +(I*cc)
            KCI_inv=np.linalg.solve(KK+0.0000001*I,I)
            S = np.dot(KCI_inv.transpose(0,2,1),KK.T).transpose(0,2,1)
            SY = np.dot(S,Y.reshape(len(Y),1))
            diag=np.dot(I*S,np.ones((len(Y),1)))-1
            err = (((Y.reshape(len(Y),1)*br-SY)/diag)**2).mean(1)
            cidx=np.argmin(err[:,0])
            self.c = c[cidx]
            self.regularization =  c[cidx]

NameError: name 'self' is not defined

In [149]:
KK = np.ones((5,5))
D, U =np.linalg.eigh(KK)
c = np.logspace(-2,2,10)
cc = c.reshape(len(c),1,1)
br = np.ones((len(c),1,1))
I = np.eye(len(KK))*br
KKK=br*KK
KCI=KKK +(I*cc)

In [150]:
KCI_inv=np.linalg.solve(KKK+0.0000001*I,I)

In [158]:
S=np.dot(KCI_inv.transpose(0,2,1),KK.T).transpose(0,2,1)
print((np.dot(S,np.ones((5,1)))-(np.ones(5).reshape(5,1)*br)).shape)

(10, 5, 1)


In [170]:
(np.ones(5).reshape(5,1)*br)
err = np.dot(I*S,np.ones((5,1))).mean(1)
np.argmin(err[:,0])

0

In [133]:
tre = np.array(([np.array([[1,2],[3,4]])*3],[np.array([[1,2],[3,4]])*2],[np.array([[1,2],[3,4]])]))
tre=tre.reshape(3,2,2)
np.dot(tre.transpose(0,2,1),np.ones((2,2))).transpose(0,2,1)

array([[[ 12.,  18.],
        [ 12.,  18.]],

       [[  8.,  12.],
        [  8.,  12.]],

       [[  4.,   6.],
        [  4.,   6.]]])

In [134]:
tre

array([[[ 3,  6],
        [ 9, 12]],

       [[ 2,  4],
        [ 6,  8]],

       [[ 1,  2],
        [ 3,  4]]])